# PricePredict Efficacy Analysis

   . Using the PricePredict class...
     . Train the model on data from 1 month in arears
     . Predict the price of the next day for the next 30 days and compare to actual price
     . Calculate the average error for each day
     
# Results...

### Apple (AAPL) Weekly Predictions
<img src='images/AAPL-Weekly 2024-12-30 15-02-23.png' alt='AAPL-Weekly Actual vs Predicted Prices' width="2000">

### Apple (AAPL) Daily Predictions
<img src='images/AAPL-Daily 2024-12-30 15-04-17.png' alt='AAPL-Daily Actual vs Predicted Prices' width="2000">

### Adobe (ADBE) Weekly Predictions
<img src='images/ADBE-Weekly 2024-12-30 15-09-49.png' alt='ADBE-Weekly Actual vs Predicted Prices' width="2000">

### Adobe (ADBE) Daily Predictions
<img src='images/ADBE-Daily 2024-12-30 15-10-36.png' alt='ADBE-Daily Actual vs Predicted Prices' width="2000">
   
### Netflix (NFLX) Weekly Predictions
<img src='images/NETFLX-Weekly 2024-12-30 14-31-27.png' alt='NETFLX-Weekly Actual vs Predicted Prices' width="2000">
 
### Netflix (NFLX) Daily Predictions
<img src='images/NETFLX-Daily 2024-12-30 14-51-44.png' alt='NETFLX-Daily Actual vs Predicted Prices' width="2000">


In [3]:
# Import Libraries
from datetime import datetime, timedelta
from pricepredict import PricePredict
import pandas as pd
import numpy as np
import logging
import warnings
warnings.filterwarnings('ignore')

from silence_tensorflow import silence_tensorflow
silence_tensorflow()
   
# Direcoty paths...
model_dir = '../models/'
chart_dir = '../charts/'
preds_dir = '../predictions/'

def prediction_analysis(ticker, period=PricePredict.PeriodDaily):
    test_ticker = "Test-" + ticker 
    pp = PricePredict(ticker,
                      period=period,
                      log_level=logging.FATAL,
                      keras_verbosity=0,
                      model_dir=model_dir,
                      chart_dir=chart_dir,
                      preds_dir=preds_dir)

    # # Turn off this objects logging to stdout
    # logger = logging.getLogger('pricepredict')
    # logger.propagate = False
    # # Turn off tensorflow logging to stdout
    # tf.get_logger().propagate = False
    
    predict_days_back = 365
    predict_days = 30
    train_days = 1000
    if period == PricePredict.PeriodWeekly:
        predict_days = 30 * 7
        predict_days_back = 365 * 7
        train_days = 2000
        
    end_train_date = datetime.now() - timedelta(days=predict_days)
    start_train_date = end_train_date - timedelta(days=train_days)
    str_end_train_date = end_train_date.strftime('%Y-%m-%d')
    str_start_train_date = start_train_date.strftime('%Y-%m-%d')
    
    pp.cache_training_data(test_ticker, str_start_train_date, str_end_train_date, period)
    
    # Create a dataframe to store the results
    df_a = pd.DataFrame(columns=['Date', 'Act_High', 'Act_Low', 'Act_Close'])
    df_p = pd.DataFrame(columns=['Date', 'Pred_High', 'Pred_Low', 'Pred_Close'])
    
    range_it = range(30, 1, -1)
    if period == PricePredict.PeriodWeekly:
        range_it = range(30 * 7, 1, -7)
        
    for days_back in range_it:
        end_pred_date = datetime.now() - timedelta(days=days_back)
        # start_pred_date = end_pred_date - timedelta(days=500)
        start_pred_date = end_pred_date - timedelta(days=predict_days_back)
        str_end_pred_date = end_pred_date.strftime('%Y-%m-%d')
        str_start_pred_date = start_pred_date.strftime('%Y-%m-%d')
        
        pp.cache_prediction_data(test_ticker, str_start_pred_date, str_end_pred_date, period)
        pp.cached_train_predict_report(no_report=True)
        
        actual_date = pp.date_data.iloc[-2]
        actual_high = round(pp.orig_data['High'].iloc[-2], 2)
        actual_low = round(pp.orig_data['Low'].iloc[-2], 2)
        actual_close = round(pp.orig_data['Adj Close'].iloc[-2], 2)

        # print(f">: actual_date: {actual_date}")
        # print(f">: Prior High: {actual_high}, Prior Low: {actual_low}, Prior Close: {actual_close}")

        # If actual_date is in the dataframe, update the actual_ data columns in the dataframe
        if not df_a['Date'].isin([actual_date]).any():
            new_row = {'Date': actual_date, 'Act_High': actual_high, 'Act_Low': actual_low, 'Act_Close': actual_close}
            df_a = pd.concat([df_a, pd.DataFrame([new_row], columns=df_a.columns)], ignore_index=True)
        
        pred_date  = pp.date_data.iloc[-1]
        pred_high  = np.round(pp.adj_pred_high [-1], 2)
        pred_low   = np.round(pp.adj_pred_low  [-1], 2)
        pred_close = np.round(pp.adj_pred_close[-1], 2)

        # print(f">: End_data: {pred_date}")
        # print(f">: Predicted High: {pred_high}, Predicted Low: {pred_low}, Predicted Close: {pred_close}")

        # Place the pred_ results in the dataframe
        if not df_p['Date'].isin([pred_date]).any():
            new_row = {'Date': pred_date, 'Pred_High': pred_high, 'Pred_Low': pred_low, 'Pred_Close': pred_close}
            df_p = pd.concat([df_p, pd.DataFrame([new_row], columns=df_p.columns)], ignore_index=True)
    
    # Drop the first 2 rows of the df_a dataframe
    df_a = df_a.iloc[1:]
    # Drop the last row of the df_p dataframe
    df_p = df_p.iloc[:-1]
    
    # Merge df_a and df_p on the Date column
    df = pd.merge(df_a, df_p, on='Date')
    # Add a column that is the absolute difference between the actual and predicted close prices
    df['Close_Diff'] = np.abs(df['Act_Close'] - df['Pred_Close'])
    # Add a column that is the absolute difference between the actual and predicted high prices
    df['High_Diff'] = np.abs(df['Act_High'] - df['Pred_High'])
    # Add a column that is the absolute difference between the actual and predicted low prices
    df['Low_Diff'] = np.abs(df['Act_Low'] - df['Pred_Low'])
    
    # Save the dataframe to a csv file
    df.to_csv(f'{test_ticker}_{period}_preds.csv', index=False, mode='w')
    
    # Calculate mean error between actual and predicted close prices
    mean_close_diff = df['Close_Diff'].mean()
    mean_high_diff = df['High_Diff'].mean()
    mean_low_diff = df['Low_Diff'].mean()
    
    # open a csv file to append the results
    fname = f'Tests_mean-err_{period}_preds.csv'
    if not os.path.exists(fname):
        with open(fname, 'w') as f:
            f.write('Ticker,Mean_High_Diff,Mean_Low_Diff,Mean_Close_Diff\n')
                    
    with open(fname, 'a') as f:
        f.write(f'{ticker}, ')    
        f.write(f'{mean_high_diff}, ')
        f.write(f'{mean_low_diff}, ')
        f.write(f'{mean_close_diff}\n')
        
    # %matplotlib inline
    # pp.gen_prediction_chart()
    
ticker = 'TSLA'
period = PricePredict.PeriodWeekly
prediction_analysis(ticker, period)
period = PricePredict.PeriodDaily
prediction_analysis(ticker, period)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [4]:
%%capture --no-stderr --no-stdout

import os

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

tickers = [ 'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'NVDA', 'PYPL', 'ADBE', 'NFLX']
for ticker in tickers: 
    period = PricePredict.PeriodDaily
    prediction_analysis(ticker, period)
    period = PricePredict.PeriodWeekly
    prediction_analysis(ticker, period)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [14]:
# Load csv file "Tests_mean-err_pred.csv" into a dataframe
df_d = pd.read_csv('Tests_mean-err_D_preds.csv')
df_w = pd.read_csv('Tests_mean-err_W_preds.csv')
# Add columns with average of the mean error columns
df_d['Mean_Diff'] = df_d[['Mean_High_Diff', 'Mean_Low_Diff', 'Mean_Close_Diff']].mean(axis=1)
df_w['Mean_Diff'] = df_w[['Mean_High_Diff', 'Mean_Low_Diff', 'Mean_Close_Diff']].mean(axis=1)

# Round the values to 2 decimal places
df_d = df_d.round(4)
df_w = df_w.round(4)

# Write the dataframes to csv files
df_d.to_csv('Tests_mean-err_D_preds.csv', index=False, mode='w')
df_w.to_csv('Tests_mean-err_W_preds.csv', index=False, mode='w')

df_d

,Ticker,Mean_High_Diff,Mean_Low_Diff,Mean_Close_Diff,Mean,Mean_Diff
0,AAPL,0.1379,0.2779,0.1168,0.1775,0.1775
1,MSFT,1.4016,1.0111,1.6668,1.3598,1.3598
2,AMZN,1.7132,1.7037,1.5563,1.6577,1.6577
3,GOOGL,0.1258,0.3595,0.2789,0.2547,0.2547
4,META,6.1911,6.2995,5.6174,6.0360,6.0360
5,TSLA,7.0932,7.2258,8.3958,7.5716,7.5716
6,NVDA,1.2389,2.3116,0.9347,1.4951,1.4951
7,PYPL,0.8474,0.9121,0.7963,0.8519,0.8519
8,ADBE,11.9284,11.2763,12.2947,11.8331,11.8331
9,NFLX,3.3495,3.5126,4.0905,3.6509,3.6509
